# Softmax 연습

*이 워크시트를 완성하고 제출하세요. (출력물과 워크시트에 포함되지 않은 코드들을 포함해서) 더 자세한 정보는 코스 웹사이트인 [숙제 페이지](http://vision.stanford.edu/teaching/cs231n/assignments.html)에서 볼 수 있습니다.*

이번 연습은 SVM과 유사합니다. 아래와 같은 것들을 하게됩니다.

- Softmax 분류기를 위한 완전히 벡터화된 **손실 함수**를 구현합니다.
- **분석 요소**를 위한 완전히 벡터화된 표현식을 구현합니다.
- 구현한것을 수치 요소로 체크합니다.
- 검증 셋을 이용해 **학습율과 정규화 강도를 튜닝**합니다.
- **SGD**를 사용해 손실 함수를 **최적화**합니다.
- 최종 학습 가중치를 **시각화**합니다.

In [ ]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# 외부 모듈의 auto-reloading을 위해 아래 링크를 확인하세요.
# http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
  """
  CIFAR-10 데이터 셋을 불러온 후 미리 준비된 선형 분류기에 전처리를 수행합니다.
  이 과정은 SVM에서 사용했던 방법과 같지만 하나의 함수로 압축되어 있습니다.
  """
  # 원시 CIFAR-10 데이터를 불러옵니다.
  cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
  X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
  
  # 데이터에서 표본을 얻습니다.
  mask = range(num_training, num_training + num_validation)
  X_val = X_train[mask]
  y_val = y_train[mask]
  mask = range(num_training)
  X_train = X_train[mask]
  y_train = y_train[mask]
  mask = range(num_test)
  X_test = X_test[mask]
  y_test = y_test[mask]
  mask = np.random.choice(num_training, num_dev, replace=False)
  X_dev = X_train[mask]
  y_dev = y_train[mask]
  
  # 전처리: 이미지 데이터를 행으로 변형합니다.
  X_train = np.reshape(X_train, (X_train.shape[0], -1))
  X_val = np.reshape(X_val, (X_val.shape[0], -1))
  X_test = np.reshape(X_test, (X_test.shape[0], -1))
  X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
  
  # 데이터 정규화: 평균 이미지 빼기
  mean_image = np.mean(X_train, axis = 0)
  X_train -= mean_image
  X_val -= mean_image
  X_test -= mean_image
  X_dev -= mean_image
  
  # 기저 차원을 더하고 열로 변형시킵니다.
  X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
  X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
  X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
  X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
  
  return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# 위 함수를 우리 데이터로 실행해봅니다.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print 'Train data shape: ', X_train.shape
print 'Train labels shape: ', y_train.shape
print 'Validation data shape: ', X_val.shape
print 'Validation labels shape: ', y_val.shape
print 'Test data shape: ', X_test.shape
print 'Test labels shape: ', y_test.shape
print 'dev data shape: ', X_dev.shape
print 'dev labels shape: ', y_dev.shape

## Softmax 분류기

**cs231n/classifiers/softmax.py**에 이번 섹션에 필요한 코드가 적혀있습니다.


In [ ]:
# 먼저 중첩 루프를 사용해 softmax 손실 함수를 구현하세요.
# cs231n/calssifiers/softmax.py 를 열고 softmax_loss_naive 함수를 구현하세요.

from cs231n.classifiers.softmax import softmax_loss_naive
import time

# 랜덤 softmax 가중치 배열을 만들고 손실을 계산하는데 사용합니다.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print 'loss: %f' % loss
print 'sanity check: %f' % (-np.log(0.1))

## 연습문제 1:
왜 손실이 -log(0.1)로 근사되는지 이유를 간단히 서술하세요.

**당신의 답:** *여기에 쓰세요*

In [ ]:
# softmax_loss_naived의 구현을 완성하고 중첩 루프를 이용한 버전을 구현해 보세요.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# SVM에서 했던 것 처럼, 수치 요소를 디버깅 툴처럼 체크해보세요.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# SVM에서처럼, 정규화를 이용해 다른 요소를 체크해보세요.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 1e2)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 1e2)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

In [ ]:
# 이제 간단하게 구현된 softmax 손실함수와 요소와 soft_max_loss_vectorized에 구현된 벡터화된 버전이 있습니다.
# 이 두가지 버전은 같은 결과를 낼 것이지만 벡터화된 버전이 좀 더 빠를것 입니다.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.00001)
toc = time.time()
print 'naive loss: %e computed in %fs' % (loss_naive, toc - tic)

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.00001)
toc = time.time()
print 'vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic)

# ASVM에서 했던것 처럼, Frobenius 방법을 사용해 두 버전의 요소를 비교할 것입니다.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print 'Loss difference: %f' % np.abs(loss_naive - loss_vectorized)
print 'Gradient difference: %f' % grad_difference

In [ ]:
# 검증셋을 이용하여 hyperparameters(정규화 강도와 학습률)를 튜닝하세요.
# 다른 범위에 대해 학습률과 정규화 강도를 실험해 보세요.
# r검증셋에 대해 0.35 이상의 분류 정확도를 얻어야 합니다.
from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 5e-7]
regularization_strengths = [5e4, 1e8]

################################################################################
# TODO:                                                                        #
# 검증셋을 이용해 학습률과 정규화 강도를 설정합니다.                           #
# 이것은 SVM에서의 검증과 같아야합니다;                                        #
# 가장 잘 학습된 softmax 분류기를 best_softmax에 저장하세요.                   #
################################################################################
pass
################################################################################
#                                  코드의 끝                                   #
################################################################################
    
# 결과를 출력합니다
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print 'lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy)
    
print 'best validation accuracy achieved during cross-validation: %f' % best_val

In [ ]:
# 테스트 셋으로 평가해 봅니다.
# 테스트 셋에서 최고의 softmax를 평가해 봅니다.
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print 'softmax on raw pixels final test set accuracy: %f' % (test_accuracy, )

In [ ]:
# 각 클래스에 대한 학습 된 가중치를 시각화
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in xrange(10):
  plt.subplot(2, 5, i + 1)
  
  # 가중치를 0과 255사이로 재조정
  wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
  plt.imshow(wimg.astype('uint8'))
  plt.axis('off')
  plt.title(classes[i])